In [2]:
pip install pandas numpy matplotlib seaborn scikit-learn xgboost

In [1]:
# Análisis de URLs de Phishing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configuración para visualizaciones
plt.style.use('ggplot')
sns.set(style="whitegrid")

In [3]:
# 1. Cargar el dataset
print("Cargando el dataset...")
try:
    # Intentar cargar el dataset (puede requerir ajustes en el delimitador)
    #df = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv')
    df = pd.read_csv("https://raw.githubusercontent.com/juancmacias/datas/main/DataSet/PhiUSIIL_Phishing_URL_Dataset.csv")
    print(f"Dataset cargado correctamente. Dimensiones: {df.shape}")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
    # Intentar con diferentes delimitadores
    try:
        df = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv', delimiter=',')
        print(f"Dataset cargado con delimitador ','. Dimensiones: {df.shape}")
    except:
        try:
            df = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv', delimiter=';')
            print(f"Dataset cargado con delimitador ';'. Dimensiones: {df.shape}")
        except Exception as e:
            print(f"No se pudo cargar el dataset: {e}")
            exit(1)

Cargando el dataset...
Dataset cargado correctamente. Dimensiones: (235795, 56)


In [5]:
# 2. Análisis Exploratorio de Datos (EDA)
print("\n--- ANÁLISIS EXPLORATORIO DE DATOS ---")

# Información básica del dataset
print("\nInformación básica del dataset:")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

# Verificar si hay una columna de etiquetas (phishing o legítimo)
print("\nColumnas en el dataset:")
print(df.columns.tolist())

# Asumiendo que la última columna es la etiqueta
target_column = df.columns[-1]
print(f"\nColumna objetivo (asumida): {target_column}")


--- ANÁLISIS EXPLORATORIO DE DATOS ---

Información básica del dataset:
Número de filas: 235795
Número de columnas: 56

Columnas en el dataset:
['FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength', 'IsDomainIP', 'TLD', 'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'Title', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOf

In [6]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df.describe())


Estadísticas descriptivas:
           URLLength   DomainLength     IsDomainIP  URLSimilarityIndex  \
count  235795.000000  235795.000000  235795.000000       235795.000000   
mean       34.573095      21.470396       0.002706           78.430778   
std        41.314153       9.150793       0.051946           28.976055   
min        13.000000       4.000000       0.000000            0.155574   
25%        23.000000      16.000000       0.000000           57.024793   
50%        27.000000      20.000000       0.000000          100.000000   
75%        34.000000      24.000000       0.000000          100.000000   
max      6097.000000     110.000000       1.000000          100.000000   

       CharContinuationRate  TLDLegitimateProb    URLCharProb      TLDLength  \
count         235795.000000      235795.000000  235795.000000  235795.000000   
mean               0.845508           0.260423       0.055747       2.764456   
std                0.216632           0.251628       0.010587    

In [7]:
# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


Valores nulos por columna:
FILENAME                      0
URL                           0
URLLength                     0
Domain                        0
DomainLength                  0
IsDomainIP                    0
TLD                           0
URLSimilarityIndex            0
CharContinuationRate          0
TLDLegitimateProb             0
URLCharProb                   0
TLDLength                     0
NoOfSubDomain                 0
HasObfuscation                0
NoOfObfuscatedChar            0
ObfuscationRatio              0
NoOfLettersInURL              0
LetterRatioInURL              0
NoOfDegitsInURL               0
DegitRatioInURL               0
NoOfEqualsInURL               0
NoOfQMarkInURL                0
NoOfAmpersandInURL            0
NoOfOtherSpecialCharsInURL    0
SpacialCharRatioInURL         0
IsHTTPS                       0
LineOfCode                    0
LargestLineLength             0
HasTitle                      0
Title                         0
DomainTitleM

In [8]:
# Distribución de la variable objetivo (si existe)
if target_column in df.columns:
    print(f"\nDistribución de la variable objetivo '{target_column}':")
    print(df[target_column].value_counts())
    print(f"Porcentaje: {df[target_column].value_counts(normalize=True) * 100}")

    # Visualización de la distribución de clases
    plt.figure(figsize=(10, 6))
    sns.countplot(x=target_column, data=df)
    plt.title('Distribución de URLs de Phishing vs Legítimas')
    plt.savefig('distribucion_clases.png')
    plt.close()


Distribución de la variable objetivo 'label':
label
1    134850
0    100945
Name: count, dtype: int64
Porcentaje: label
1    57.189508
0    42.810492
Name: proportion, dtype: float64


In [9]:
# Matriz de correlación
print("\nCalculando matriz de correlación...")
try:
    # Seleccionar solo columnas numéricas
    numeric_df = df.select_dtypes(include=[np.number])

    # Calcular y visualizar la matriz de correlación
    plt.figure(figsize=(20, 16))
    correlation_matrix = numeric_df.corr()
    mask = np.triu(correlation_matrix)
    sns.heatmap(correlation_matrix, annot=False, mask=mask, cmap='coolwarm', linewidths=0.5)
    plt.title('Matriz de Correlación de Características')
    plt.tight_layout()
    plt.savefig('matriz_correlacion.png')
    plt.close()

    # Identificar características altamente correlacionadas
    high_corr_features = []
    corr_threshold = 0.8

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > corr_threshold:
                high_corr_features.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

    if high_corr_features:
        print("\nCaracterísticas altamente correlacionadas (|corr| > 0.8):")
        for feat1, feat2, corr in high_corr_features:
            print(f"{feat1} - {feat2}: {corr:.4f}")
    else:
        print("\nNo se encontraron características altamente correlacionadas (|corr| > 0.8)")

except Exception as e:
    print(f"Error al calcular la matriz de correlación: {e}")


Calculando matriz de correlación...

Características altamente correlacionadas (|corr| > 0.8):
NoOfLettersInURL - URLLength: 0.9560
NoOfDegitsInURL - URLLength: 0.8358
NoOfEqualsInURL - NoOfDegitsInURL: 0.8060
URLTitleMatchScore - DomainTitleMatchScore: 0.9610
label - URLSimilarityIndex: 0.8604


In [10]:
# Histogramas de características numéricas
try:
    numeric_columns = numeric_df.columns[:10]  # Limitar a 10 columnas para no generar demasiados gráficos
    plt.figure(figsize=(20, 15))
    for i, column in enumerate(numeric_columns, 1):
        plt.subplot(3, 4, i)
        sns.histplot(df[column], kde=True)
        plt.title(f'Distribución de {column}')
    plt.tight_layout()
    plt.savefig('histogramas_caracteristicas.png')
    plt.close()
except Exception as e:
    print(f"Error al generar histogramas: {e}")

In [12]:
# 3. Preparación de datos para modelado
print("\n--- PREPARACIÓN DE DATOS PARA MODELADO ---")

# Separar características y variable objetivo
#if target_column in df.columns:
X = df.drop(columns=[target_column])
y = df[target_column]

# Eliminar columnas no numéricas o irrelevantes para el modelado
non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"\nEliminando columnas no numéricas: {non_numeric_cols}")
    X = X.select_dtypes(include=[np.number])

# Verificar si hay valores infinitos y reemplazarlos
X = X.replace([np.inf, -np.inf], np.nan)

# Imputar valores nulos con la media
if X.isnull().sum().sum() > 0:
    print(f"\nImputando {X.isnull().sum().sum()} valores nulos con la media")
    X = X.fillna(X.mean())

# Normalizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nConjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de prueba: {X_test.shape[0]} muestras")


--- PREPARACIÓN DE DATOS PARA MODELADO ---

Eliminando columnas no numéricas: ['FILENAME', 'URL', 'Domain', 'TLD', 'Title']

Conjunto de entrenamiento: 188636 muestras
Conjunto de prueba: 47159 muestras


In [13]:
# 4. Modelado y Evaluación
print("\n--- MODELADO Y EVALUACIÓN ---")

# Lista para almacenar resultados de modelos
models_results = []

# Función para entrenar y evaluar modelos
def train_evaluate_model(model, model_name, X_train, X_test, y_train, y_test):
    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicciones
    y_pred = model.predict(X_test)

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)

    # Guardar resultados
    results = {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm
    }

    # Validación cruzada
    cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    results['cv_mean'] = cv_scores.mean()
    results['cv_std'] = cv_scores.std()

    # Calcular overfitting
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    results['train_accuracy'] = train_accuracy
    results['overfitting'] = train_accuracy - accuracy

    return results


--- MODELADO Y EVALUACIÓN ---


In [14]:
# Modelo 1: Random Forest
print("\nEntrenando Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_results = train_evaluate_model(rf_model, "Random Forest", X_train, X_test, y_train, y_test)
models_results.append(rf_results)

# Modelo 2: XGBoost
print("\nEntrenando XGBoost...")
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_results = train_evaluate_model(xgb_model, "XGBoost", X_train, X_test, y_train, y_test)
models_results.append(xgb_results)

# Modelo 3: Regresión Logística
print("\nEntrenando Regresión Logística...")
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_results = train_evaluate_model(lr_model, "Regresión Logística", X_train, X_test, y_train, y_test)
models_results.append(lr_results)

# Modelo 4: SVM
print("\nEntrenando SVM...")
svm_model = SVC(random_state=42)
svm_results = train_evaluate_model(svm_model, "SVM", X_train, X_test, y_train, y_test)
models_results.append(svm_results)

# Modelo 5: Árbol de Decisión
print("\nEntrenando Árbol de Decisión...")
dt_model = DecisionTreeClassifier(random_state=42)
dt_results = train_evaluate_model(dt_model, "Árbol de Decisión", X_train, X_test, y_train, y_test)
models_results.append(dt_results)

# Resumen de resultados
print("\n--- RESUMEN DE RESULTADOS ---")
results_df = pd.DataFrame()

for result in models_results:
    model_df = pd.DataFrame({
        'Modelo': [result['model_name']],
        'Accuracy': [result['accuracy']],
        'Precision': [result['precision']],
        'Recall': [result['recall']],
        'F1-Score': [result['f1_score']],
        'CV Mean': [result['cv_mean']],
        'CV Std': [result['cv_std']],
        'Train Accuracy': [result['train_accuracy']],
        'Overfitting': [result['overfitting']]
    })
    results_df = pd.concat([results_df, model_df], ignore_index=True)

print("\nComparación de modelos:")
print(results_df.to_string(index=False))

# Visualización de resultados
plt.figure(figsize=(12, 8))


Entrenando Random Forest...

Entrenando XGBoost...

Entrenando Regresión Logística...

Entrenando SVM...

Entrenando Árbol de Decisión...

--- RESUMEN DE RESULTADOS ---

Comparación de modelos:
             Modelo  Accuracy  Precision   Recall  F1-Score  CV Mean   CV Std  Train Accuracy  Overfitting
      Random Forest  1.000000   1.000000 1.000000  1.000000 1.000000 0.000000        1.000000     0.000000
            XGBoost  1.000000   1.000000 1.000000  1.000000 0.999996 0.000008        1.000000     0.000000
Regresión Logística  0.999873   0.999873 0.999873  0.999873 0.999873 0.000040        0.999894     0.000021
                SVM  0.999724   0.999724 0.999724  0.999724 0.999763 0.000078        0.999926     0.000201
  Árbol de Decisión  1.000000   1.000000 1.000000  1.000000 1.000000 0.000000        1.000000     0.000000


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [17]:
# Gráfico de barras para accuracy
plt.subplot(2, 2, 1)
sns.barplot(x='Modelo', y='Accuracy', data=results_df)
plt.title('Accuracy por Modelo')
plt.xticks(rotation=45)

# Gráfico de barras para F1-Score
plt.subplot(2, 2, 2)
sns.barplot(x='Modelo', y='F1-Score', data=results_df)
plt.title('F1-Score por Modelo')
plt.xticks(rotation=45)

# Gráfico de barras para overfitting
plt.subplot(2, 2, 3)
sns.barplot(x='Modelo', y='Overfitting', data=results_df)
plt.title('Overfitting por Modelo')
plt.xticks(rotation=45)

# Gráfico de barras para validación cruzada
plt.subplot(2, 2, 4)
sns.barplot(x='Modelo', y='CV Mean', data=results_df)
plt.title('Validación Cruzada por Modelo')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('comparacion_modelos.png')
plt.close()

# Identificar el mejor modelo
best_model_idx = results_df['F1-Score'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Modelo']
print(f"\nEl mejor modelo basado en F1-Score es: {best_model_name}")

# Análisis de características importantes (para Random Forest)
if 'Random Forest' in results_df['Modelo'].values:
    print("\n--- ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS (RANDOM FOREST) ---")
    feature_importances = rf_model.feature_importances_
    feature_names = X.columns

    # Crear DataFrame para visualización
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    }).sort_values('Importance', ascending=False)

    print("\nTop 10 características más importantes:")
    print(importance_df.head(10).to_string(index=False))

    # Visualización de importancia de características
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(15))
    plt.title('Top 15 Características Más Importantes (Random Forest)')
    plt.tight_layout()
    plt.savefig('importancia_caracteristicas.png')
    plt.close()


El mejor modelo basado en F1-Score es: Random Forest

--- ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS (RANDOM FOREST) ---

Top 10 características más importantes:
           Feature  Importance
URLSimilarityIndex    0.181075
   NoOfExternalRef    0.169265
        LineOfCode    0.146585
       NoOfSelfRef    0.108536
         NoOfImage    0.084350
            NoOfJS    0.072904
           NoOfCSS    0.031598
      HasSocialNet    0.031526
  HasCopyrightInfo    0.025522
           IsHTTPS    0.022940


In [18]:
# Optimización de hiperparámetros para el mejor modelo
print("\n--- OPTIMIZACIÓN DE HIPERPARÁMETROS ---")

if best_model_name == "Random Forest":
    print("\nOptimizando hiperparámetros para Random Forest...")
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)

    print(f"\nMejores parámetros: {grid_search.best_params_}")
    print(f"Mejor F1-Score: {grid_search.best_score_:.4f}")

    # Evaluar modelo optimizado
    best_rf = grid_search.best_estimator_
    y_pred_optimized = best_rf.predict(X_test)

    print("\nRendimiento del modelo optimizado:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_optimized):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_optimized, average='weighted'):.4f}")

    # Verificar overfitting
    train_acc_opt = accuracy_score(y_train, best_rf.predict(X_train))
    test_acc_opt = accuracy_score(y_test, y_pred_optimized)
    overfitting = train_acc_opt - test_acc_opt

    print(f"\nAccuracy en entrenamiento: {train_acc_opt:.4f}")
    print(f"Accuracy en prueba: {test_acc_opt:.4f}")
    print(f"Overfitting: {overfitting:.4f}")

    if overfitting > 0.05:
        print("\n¡ADVERTENCIA! El modelo presenta un overfitting superior al 5%.")
        print("Recomendaciones para reducir el overfitting:")
        print("1. Aumentar la regularización")
        print("2. Reducir la complejidad del modelo")
        print("3. Utilizar técnicas de early stopping")
        print("4. Aplicar dropout o bagging")

elif best_model_name == "XGBoost":
    print("\nOptimizando hiperparámetros para XGBoost...")
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    }
    grid_search = GridSearchCV(xgb.XGBClassifier(random_state=42), param_grid, cv=3, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)

    print(f"\nMejores parámetros: {grid_search.best_params_}")
    print(f"Mejor F1-Score: {grid_search.best_score_:.4f}")

    # Evaluar modelo optimizado
    best_xgb = grid_search.best_estimator_
    y_pred_optimized = best_xgb.predict(X_test)

    print("\nRendimiento del modelo optimizado:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_optimized):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_optimized, average='weighted'):.4f}")

    # Verificar overfitting
    train_acc_opt = accuracy_score(y_train, best_xgb.predict(X_train))
    test_acc_opt = accuracy_score(y_test, y_pred_optimized)
    overfitting = train_acc_opt - test_acc_opt

    print(f"\nAccuracy en entrenamiento: {train_acc_opt:.4f}")
    print(f"Accuracy en prueba: {test_acc_opt:.4f}")
    print(f"Overfitting: {overfitting:.4f}")

    if overfitting > 0.05:
        print("\n¡ADVERTENCIA! El modelo presenta un overfitting superior al 5%.")
        print("Recomendaciones para reducir el overfitting:")
        print("1. Aumentar la regularización (alpha, lambda)")
        print("2. Reducir la profundidad del árbol")
        print("3. Disminuir la tasa de aprendizaje")
        print("4. Aumentar el parámetro de subsample")

# Guardar resultados en un archivo
results_df.to_csv('resultados_modelos.csv', index=False)
print("\nResultados guardados en 'resultados_modelos.csv'")

print("\n--- ANÁLISIS COMPLETADO ---")
#else:
    #print(f"\nNo se pudo identificar la columna objetivo en el dataset.")


--- OPTIMIZACIÓN DE HIPERPARÁMETROS ---

Optimizando hiperparámetros para Random Forest...

Mejores parámetros: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Mejor F1-Score: 1.0000

Rendimiento del modelo optimizado:
Accuracy: 1.0000
F1-Score: 1.0000

Accuracy en entrenamiento: 1.0000
Accuracy en prueba: 1.0000
Overfitting: 0.0000

Resultados guardados en 'resultados_modelos.csv'

--- ANÁLISIS COMPLETADO ---
